In [1]:
import os
import re
import random
import requests
import pandas as pd
from io import BytesIO
from queue import Queue
from urllib.parse import urlparse

from bs4 import BeautifulSoup
from newspaper import Article
import nltk
import sqlalchemy
import psycopg2
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

from sklearn.metrics.pairwise import cosine_similarity
import pyodbc

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from PIL import Image
prompt="You will given scrapped data that may or may not be in proper format. Find out the portion that is about the news and generate a 70-80 word sumary of it \n .News:"
from sqlalchemy import create_engine
import os
os.environ['MISTRAL_API_KEY']='YTqR9jptXj0vG8LbHhaPsRfJkNkBB9Xi'
import pyodbc,socket
import urllib
from sqlalchemy import text
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-tiny"
server_ip = '127.0.0.1'  # Replace with the actual IP address of your SQL Server
port = '1433'
server = 'Butt_Sahab\SQLEXPRESS01'
database = 'Scrapped_News'  # Replace with your database name
username = 'King'
password = 'Dio'
driver = '{ODBC Driver 17 for SQL Server}'
connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
client = MistralClient(api_key=api_key)
params = urllib.parse.quote_plus(connection_string)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
import numpy as np
#nltk.download('popular')

In [2]:
def extract_largest_srcset_image(srcset):
    # Split the string into parts on commas
    parts = srcset.split(',')
    largest_src = None
    max_width = 0
    for part in parts:
        # Each part is expected to be in format "url width", e.g., "example.jpg 600w"
        match = re.search(r'(\S+) (\d+)w', part.strip())
        if match:
            src, width = match.groups()
            width = int(width)
            if width > max_width:
                largest_src = src
                max_width = width
    return largest_src, max_width

def extract_dimensions_from_url(url):
    # Regex pattern to match dimensions in the URL (e.g., "resize/970x647" or "responsive640")
    pattern = re.compile(r'(\d+)x(\d+)|responsive(\d+)')
    match = pattern.search(url)
    
    if match:
        if match.group(3):  # Matched the "responsive" pattern, width only
            width = int(match.group(3))
            height = width  # Placeholder, actual height depends on aspect ratio
        else:  # Matched the "WxH" pattern
            width, height = int(match.group(1)), int(match.group(2))
        return width, height
    return None, None

def extract_dimensions_somehow(img):
    width = height = None
    # Try to extract from attributes
    if 'width' in img.attrs and 'height' in img.attrs and img.attrs['width']!='' and img.attrs['height']!='':
        width = float(img['width'].replace('px', ''))
        height = float(img['height'].replace('px', ''))
    # Try to extract from srcset
    elif 'srcset' in img.attrs:
        _, width = extract_largest_srcset_image(img['srcset'])
        height = width # Assuming height based on aspect ratio
    # Backup method: Try to extract from image URL
    if width is None:
        src = img.get('src', '')
        width, height = extract_dimensions_from_url(src)
    return width, height


from urllib.parse import urlparse

def get_base_url(url):
    parsed_url = urlparse(url)
    # Construct the base URL using scheme and netloc
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    return base_url

from datetime import datetime, timedelta

def convert_to_datetime(date_str):
    now = datetime.now()
    if date_str.strip().lower() == 'live':
        return now
    try:
        date_str=date_str.strip().lower()
        if 'ago' in date_str:
            number, unit = date_str.split()[:2]
            number = int(number)
            unit = unit.rstrip('s').lower()  # Normalize the unit to handle both singular and plural
            if unit == 'min':
                return now - timedelta(minutes=number)
            elif unit == 'hour':
                return now - timedelta(hours=number)
            elif unit == 'day':
                return now - timedelta(days=number)
            elif unit == 'week':
                return now - timedelta(weeks=number)
            elif unit == 'month':
                # Approximation: assumes 30 days in a month
                return now - timedelta(days=number*30)
            else:
                # Log the unhandled unit
                print(f"Unhandled unit in date string: {date_str}")
                return None
        else:
            # Attempt to parse as a standard date format
            return pd.to_datetime(date_str, errors='coerce')  # 'coerce' will set unparseable dates to NaT
    except ValueError as e:
        # Log the error and the problematic date string
        print(f"Error parsing date string '{date_str}': {e}")
        return None
def genrate_summary(soup_text,article_text):
    if (soup_text=="" and article_text==""):
        return 'Empty'
    if (len(article_text)>len(soup_text)):
        if len(article_text.split(' '))>=8000:
            return 'Too many Tokens'
        messages = [
        ChatMessage(role="user", content=prompt+article_text)
    ]
    else:
        if len(soup_text.split(' '))>=8000:
            return 'Too many Tokens'
        messages = [
            ChatMessage(role="user", content=prompt+soup_text)
        ]
    chat_response = client.chat(
        model=model,
        messages=messages,
    )
    return chat_response.choices[0].message.content.strip()
def df_to_db(df):
    # Assuming df is your DataFrame and it's potentially large

    with engine.begin() as conn:
        # Step 1: Insert the DataFrame into a staging table
        df.to_sql('StagingTable', con=conn, if_exists='replace', index=False, method='multi')

        # Step 2: Use MERGE to insert new records from the staging table to the target table
        merge_query = """
        MERGE INTO News AS target
        USING StagingTable AS source
        ON target.url = source.url
        WHEN NOT MATCHED BY TARGET THEN
            INSERT (url, article_text, soup_text, article_title, date, top_image_url, Location, Summary)
            VALUES (source.url, source.article_text, source.soup_text, source.article_title, source.date, source.top_image_url, source.Location, source.Summary);
        """
        conn.execute(text(merge_query))

        # Optional: Clean up the staging table after use
        conn.execute(text("DROP TABLE StagingTable;"))

def return_image(soup,base_url):
    valid_images = []
    if 'https://www.dailymail.co.uk/' in base_url:
        image_elements=soup.find_all('image-wrap fff-pic')
    else:
        image_elements=soup.find_all('img')            
    if image_elements==[]:
        image_elements=soup.find_all('app-c-figure__image')
    #print(image_elements)
    valid_images = []
    for img in image_elements:
        src = img.get("src")
        if src:
            # Skip SVG images
            if 'svg' in src or src.endswith('.svg'):
                continue
            if not src.startswith(('http:', 'https:')):
                src = base_url + src
            width, height = extract_dimensions_somehow(img)
            if width and height:
                area = width * height
                valid_images.append({"src": src, "width": width, "height": height, "area": area})

    valid_images = [image for image in valid_images if not 'data:image/' in image['src']]
            # Sort the valid images by area in descending order
    sorted_images = sorted(valid_images, key=lambda x: x["area"], reverse=True)
    
        # Get the top 4 images with the largest areas
    top_img = sorted_images[:1]
    # if top_img==[]:
    #     top_img = image_elements[0]
    return top_img

In [3]:
def scrape_and_process(url,Location):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(16)
    # Initialize a list to store the results
    results = []
    try:
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        googlelnks = soup.find_all('a', 'WlydOe')
        urls = [link.get('href') for link in googlelnks]
        tests= soup.find_all('div','OSrXXb rbYSKb LfVVr')
        dates = []
        #article = Article('')
        # Iterate through each <div> element and extract the text within the <span> tag
        for div in tests:
            span = div.find('span')  # Find the <span> tag within the <div>
            if span:  # Check if the <span> tag exists
                dates.append(span.text) 
        for url, date in zip(urls, dates):
            driver.set_page_load_timeout(16)
            try:
                driver.get(url)
                driver_html=driver.page_source
            except:
                driver_html='None'
            base_url = get_base_url(url)
            if 'youtube.com' in url :
                continue
            print(url)
            try:
                print('getting response')
                response = requests.get(url, timeout=10)  # 10 seconds timeout
                html = response.content
            except:
                print('Getting response failed')
                try:
                    print("getting response failed now getitng article with url")
                    article = Article(url)
                    article.build()
                except:
                    print("getting response failed article with url failed using driver.page_source")
                    article = Article('')
                    article.set_html(driver_html)
                    article.parse()
                article_text = article.text
                article_title = article.title
                soup = BeautifulSoup(article.html)
                top_img = return_image(soup,base_url)
                if top_img==[] or (len(article_text.split(' '))<200 and len(soup.text.split(' '))<200):
                    continue
                results.append({
                'url':url,
                'article_text': article_text.replace('\n', ' '),
                'soup_text': soup.text,
                'top_image_url': top_img[0]['src'],
                'article_title': article_title,
                'date':convert_to_datetime(date),
                'Location':Location,
                'Summary':genrate_summary(soup.text,article_text)
            })
                continue
                #soup = BeautifulSoup(html, 'html.parser')
                #soup_text = soup.get_text(separator=' ', strip=True).replace('\n', ' ')
            try:
                print("reponse passed getting article from url ")
                article = Article(url)
                article.build()
            except:
                print(" reponse passed getting article from url failed using page_source")
                article = Article('')
                article.set_html(driver_html)
                if(driver_html=='None'):
                    continue
                article.parse()
                #print(article.text)
            if(len(html)<400):
                html=article.html
            article_text = article.text
            article_title = article.title
            if 'nytimes.com'  in base_url:
                html=article.html
            
            soup = BeautifulSoup(html)
            top_img = return_image(soup,base_url)
            if top_img==[] or (len(article_text.split(' '))<200 and len(soup.text.split(' '))<200):
                continue
            #print(article_text.replace('\n', ' '))
            results.append({
            'url':url,
            'article_text': article_text.replace('\n', ' '),
            'soup_text': soup.text,
            'top_image_url': top_img[0]['src'],
            'article_title': article_title    ,
            'date':convert_to_datetime(date),
            'Location':Location,
            'Summary':genrate_summary(soup.text,article_text)        
            })
            continue
    except Exception as e:
        print("Url failed")
        print(e)
        return 
    finally:
        driver.quit()

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    #print(df)
    try:
        df_to_db(df)
    except:
        print(df)

In [4]:
for num in range(0, 300, 10):
    print(num)
    uk_urls=[
         f'https://www.google.co.uk/search?q=DAILY+MAIL+UK+NEWS&gl=uk&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=erG7Za6bD6GGxc8PmO-qqAg&ved=0ahUKEwiuwN7Gs4qEAxUhQ_EDHZi3CoUQ4dUDCA4&oq=DAILY+MAIL+UK+NEWS&gs_lp=Egxnd3Mtd2l6LW5ld3MiEkRBSUxZIE1BSUwgVUsgTkVXUzIFEAAYgAQyCxAAGIAEGIoFGJECMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeSJGUAVDAAljeiQFwAXgAkAEAmAHNAqAB0xCqAQUyLTIuNbgBDMgBAPgBAvgBAcICEBAAGIAEGIoFGEMYsQMYgwHCAgoQABiABBiKBRhDwgIGEAAYFhgewgIKEAAYCBgHGB4YCsICBBAAGB6IBgE&sclient=gws-wiz-news&start={num}',
         f'https://www.google.co.uk/search?q=Telegraph+Uk+News&gl=uk&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=lrG7ZZyIHPiOxc8P3taPgAY&ved=0ahUKEwicq5jUs4qEAxV4R_EDHV7rA2AQ4dUDCA4&uact=5&oq=Telegraph+Uk+News&gs_lp=Egxnd3Mtd2l6LW5ld3MiEVRlbGVncmFwaCBVayBOZXdzMgcQABiABBgNMgcQABiABBgNMgcQABiABBgNMgcQABiABBgNMgcQABiABBgNMgcQABiABBgNMgcQABiABBgNMgcQABiABBgNMgcQABiABBgNMgcQABiABBgNSN-sAVAAWLSrAXADeACQAQCYAccDoAGQS6oBCTItMTAuMTkuMbgBA8gBAPgBAvgBAcICBRAAGIAEwgIKEAAYgAQYigUYQ8ICBhAAGAcYHsICBBAAGB7CAgYQABgFGB7CAgsQABiABBiKBRiRAsICChAAGAcYHhgPGArCAgoQABgFGAcYHhgPwgIIEAAYBRgHGB7CAgYQABgIGB7CAggQIRigARjDBMICCBAAGIAEGKIEwgIGEAAYFhgewgILEAAYgAQYigUYhgPCAggQABgWGB4YD8ICCBAAGBYYHhgKwgIFECEYoAHCAgcQABiABBgKiAYB&sclient=gws-wiz-news&start={num}',
         f'https://www.google.co.uk/search?q=The+Guardian+uk+news&gl=uk&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=IrK7ZfHxF9yqxc8PksKLgAg&ved=0ahUKEwjxivWWtIqEAxVcVfEDHRLhAoAQ4dUDCA4&uact=5&oq=The+Guardian+uk+news&gs_lp=Egxnd3Mtd2l6LW5ld3MiFFRoZSBHdWFyZGlhbiB1ayBuZXdzMgsQABiABBiKBRiRAjILEAAYgAQYigUYkQIyCxAAGIAEGIoFGJECMgUQABiABDIFEAAYgAQyBRAAGIAEMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeMgYQABgWGB5I_sQBUABY08MBcAN4AJABAJgB9AOgAbI0qgEKMi05LjExLjAuMbgBA8gBAPgBAvgBAcICChAAGIAEGIoFGEPCAhAQABiABBiKBRhDGLEDGIMBwgILEAAYgAQYsQMYgwHCAhEQABiABBiKBRiRAhixAxiDAcICDRAAGIAEGAoYsQMYgwHCAgoQABiABBgKGLEDwgIHEAAYgAQYCsICCBAAGIAEGLEDwgIEEAAYA8ICDhAAGIAEGIoFGLEDGIMBwgILEAAYgAQYigUYhgOIBgE&sclient=gws-wiz-news&start={num}',
         f'https://www.google.co.uk/search?q=BBC+news+uk&gl=uk&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=aLK7Zci8NKzAxc8P9raDkA8&ved=0ahUKEwjIkMK4tIqEAxUsYPEDHXbbAPIQ4dUDCA4&uact=5&oq=BBC+news+uk&gs_lp=Egxnd3Mtd2l6LW5ld3MiC0JCQyBuZXdzIHVrMhEQABiABBiKBRiRAhixAxiDATILEAAYgAQYigUYkQIyCxAAGIAEGIoFGJECMgsQABiABBiKBRiRAjIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgsQABiABBiKBRiRAkiGMFAAWKEqcAF4AJABAJgByQKgAYIaqgEFMi01Lja4AQPIAQD4AQHCAgcQABiABBgKwgIKEAAYgAQYigUYQ8ICBhAAGAcYHsICBhAAGB4YD8ICBhAAGAgYHsICCBAAGAgYHhgKwgILEAAYgAQYsQMYgwE&sclient=gws-wiz-news&start={num}',
         f'https://www.google.co.uk/search?q=Independent+co.uk+news&gl=uk&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=FbO7ZcyzIbTYxc8P1rSt6Aw&ved=0ahUKEwiMku6KtYqEAxU0bPEDHVZaC80Q4dUDCA4&uact=5&oq=Independent+co.uk+news&gs_lp=Egxnd3Mtd2l6LW5ld3MiFkluZGVwZW5kZW50IGNvLnVrIG5ld3MyBhAAGBYYHjIGEAAYFhgeMgYQABgWGB4yCxAAGIAEGIoFGIYDSJeBAVC8AliLgAFwAXgAkAEBmAGnA6ABmDOqAQkyLTEwLjEwLjG4AQPIAQD4AQL4AQHCAhAQABiABBiKBRhDGLEDGIMBwgIFEAAYgATCAgoQABiABBiKBRhDwgILEAAYgAQYsQMYgwHCAg4QABiABBiKBRixAxiDAcICDRAAGIAEGIoFGEMYsQPCAggQABiABBixA8ICBBAAGAPCAgcQABiABBgKwgIHEAAYgAQYDYgGAQ&sclient=gws-wiz-news&start={num}'
    ]
    usa_urls=[
         f'https://www.google.com/search?q=nytimes+news+&gl=us&hl=en&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&sxsrf=ACQVn086Se-kigRJyeZExz-6p29YfS6uyw%3A1706800026751&ei=mrO7ZYayLeyExc8P_7iwqAg&ved=0ahUKEwiG56_KtYqEAxVsQvEDHX8cDIUQ4dUDCA4&uact=5&oq=nytimes+news+&gs_lp=Egxnd3Mtd2l6LW5ld3MiDW55dGltZXMgbmV3cyAyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgcQABiABBgKMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgARIxlVQ9ApYtkBwAngAkAEBmAGTA6ABgyiqAQgyLTMuMTIuMbgBA8gBAPgBAcICEBAAGIAEGIoFGEMYsQMYgwHCAgoQABiABBiKBRhDwgILEAAYgAQYsQMYgwHCAg4QABiABBiKBRixAxiDAcICDRAAGIAEGIoFGEMYsQPCAhEQABiABBiKBRiRAhixAxiDAYgGAQ&sclient=gws-wiz-news&start={num}',
         f'https://www.google.com/search?q=cnn.com+news&gl=us&hl=en&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&sxsrf=ACQVn08fqtT2pMPRGkECd2uNqduUpPws-Q%3A1706800173787&ei=LbS7ZYXRL5yUxc8P15276AY&ved=0ahUKEwjFm76QtoqEAxUcSvEDHdfODm0Q4dUDCA4&uact=5&oq=cnn.com+news&gs_lp=Egxnd3Mtd2l6LW5ld3MiDGNubi5jb20gbmV3czIQEAAYgAQYigUYQxixAxiDATILEAAYgAQYigUYkQIyDhAAGIAEGIoFGJECGLEDMgsQABiABBiKBRiRAjILEAAYgAQYigUYkQIyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABEjIDlAAWKwMcAB4AJABAJgBzwKgAekOqgEDMy02uAEDyAEA-AEBwgIHEAAYgAQYCg&sclient=gws-wiz-news&start={num}',
         f'https://www.google.com/search?q=usatoday.com+news&gl=us&hl=en&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&sxsrf=ACQVn09qfL0jnh-kyJnkoAxX-ZzwTZmCUQ%3A1706800052431&ei=tLO7ZfPmGfSJxc8Pgq-58AU&ved=0ahUKEwjzkM_WtYqEAxX0RPEDHYJXDl4Q4dUDCA4&uact=5&oq=usatoday.com+news&gs_lp=Egxnd3Mtd2l6LW5ld3MiEXVzYXRvZGF5LmNvbSBuZXdzMgsQABiABBiKBRiRAjIFEAAYgAQyCxAAGIAEGIoFGIYDSPInUJwDWIkncAF4AJABAJgB3AKgAeQmqgEGMi0yLjE0uAEDyAEA-AEC-AEBwgIKEAAYgAQYigUYQ8ICCxAAGIAEGLEDGIMBwgIOEAAYgAQYigUYsQMYgwHCAhAQABiABBiKBRhDGLEDGIMBwgIEEAAYA8ICCBAAGIAEGLEDwgIHEAAYgAQYCsICCBAAGBYYHhgPwgIGEAAYFhgeiAYB&sclient=gws-wiz-news&start={num}',
         f'https://www.google.com/search?q=dailymail.com+news&gl=us&hl=en&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&sxsrf=ACQVn09QJq8il7jCKdLkzAbGteJ9BGyqyQ%3A1706800229087&ei=ZbS7ZfL0BOm_xc8PsZSnmAI&ved=0ahUKEwiyu-2qtoqEAxXpX_EDHTHKCSMQ4dUDCA4&uact=5&oq=dailymail.com+news&gs_lp=Egxnd3Mtd2l6LW5ld3MiEmRhaWx5bWFpbC5jb20gbmV3czIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIIEAAYBxgeGAoyBBAAGB4yCBAAGAUYBxgeMggQABgFGAcYHjIGEAAYCBgeMgYQABgIGB4yBhAAGAgYHkiZDVDNCFjbC3AAeACQAQGYAdQDoAG6C6oBBTMtMy4xuAEDyAEA-AEBiAYB&sclient=gws-wiz-news&start={num}',
         f'https://www.google.com/search?q=cbsnews.com+news&gl=us&hl=en&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&sxsrf=ACQVn0--TP53TVH9bzsrxclImGDEPPtiyA%3A1706800245251&ei=dbS7ZeT3DoKRxc8P3pSGwAM&ved=0ahUKEwikhsiytoqEAxWCSPEDHV6KATgQ4dUDCA4&uact=5&oq=cbsnews.com+news&gs_lp=Egxnd3Mtd2l6LW5ld3MiEGNic25ld3MuY29tIG5ld3MyBRAAGIAEMgUQABiABEiEYVDRA1iyX3AGeACQAQCYAZoDoAGLNaoBCDItMy4xNy4xuAEDyAEA-AEC-AEBwgIIEAAYCBgeGArCAgYQABgIGB7CAgsQABiABBiKBRiGA8ICBhAAGBYYHsICCBAAGBYYHhgKwgILEAAYgAQYigUYkQLCAggQABiABBixA8ICCxAAGIAEGLEDGIMBwgIOEAAYgAQYigUYsQMYgwHCAgoQABiABBiKBRhDwgIREAAYgAQYigUYkQIYsQMYgwHCAhAQABiABBiKBRhDGLEDGIMBwgIOEAAYgAQYigUYkQIYsQPCAgcQABiABBgKwgIMEAAYgAQYigUYQxgKiAYB&sclient=gws-wiz-news&start={num}'
    ]
    australia_urls=[
         f'https://www.google.com.au/search?q=heraldsun.com.au+news&gl=au&hl=en&tbs=qdr:d,sbd:1&tbm=nws&ei=4rm7Zf869IGL6A-_qJyACQ&start=0&sa=N&ved=2ahUKEwj_s-PIu4qEAxX0wAIHHT8UB5A4KBDy0wN6BAgFEAQ&biw=1920&bih=911&dpr=1&start={num}',
         f'https://www.google.com.au/search?q=theaustralian.com.au+news&gl=au&hl=en&biw=1920&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=5Lm7ZbauHZ3ui-gPl7CMwAE&ved=0ahUKEwi2sPrJu4qEAxUd9wIHHRcYAxgQ4dUDCA0&uact=5&oq=theaustralian.com.au+news&gs_lp=Egxnd3Mtd2l6LW5ld3MiGXRoZWF1c3RyYWxpYW4uY29tLmF1IG5ld3MyBhAAGAUYHjILEAAYgAQYigUYhgMyCxAAGIAEGIoFGIYDMgsQABiABBiKBRiGA0j6lgdQs80GWOuTB3AAeACQAQCYAf0DoAGXP6oBCjItNC4xMS42LjG4AQPIAQD4AQHCAgYQABgWGB7CAhAQABiABBiKBRhDGLEDGIMBwgILEAAYgAQYigUYkQLCAgsQABiABBixAxiDAcICCBAAGIAEGLEDwgIFEAAYgATCAgYQABgHGB7CAggQABgHGB4YCogGAQ&sclient=gws-wiz-news&start={num}',
         f'https://www.google.com.au/search?q=the+sydney+morning+herald&gl=au&hl=en&tbs=qdr:d,sbd:1&tbm=nws&ei=k7q7ZfHqE6Lpi-gPo-afuAI&start=0&sa=N&ved=2ahUKEwixgKqdvIqEAxWi9AIHHSPzByc4WhDy0wN6BAgEEAQ&biw=1920&bih=911&dpr=1&start={num}',
         f'https://www.google.com.au/search?q=the+age.com.au+&gl=au&hl=en&biw=1920&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=lrq7ZeipB9-ci-gPyMeZiAI&ved=0ahUKEwjozNSevIqEAxVfzgIHHchjBiEQ4dUDCA0&uact=5&oq=the+age.com.au+&gs_lp=Egxnd3Mtd2l6LW5ld3MiD3RoZSBhZ2UuY29tLmF1IDIHEAAYgAQYDTIHEAAYgAQYDTIHEAAYgAQYDTIHEAAYgAQYDTIHEAAYgAQYDTIHEAAYgAQYDTIGEAAYFhgeMgYQABgWGB5I7SdQxwVYvRdwAHgAkAEBmAHpBKABtieqAQkyLTUuMi41LjG4AQPIAQD4AQHCAgUQABiABMICChAAGIAEGIoFGEPCAggQABiABBixA8ICCxAAGIAEGLEDGIMBiAYB&sclient=gws-wiz-news&start={num}',
         f'https://www.google.com.au/search?q=9+news+australia&gl=au&hl=en&biw=1920&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=prq7ZfWeBIWYi-gP8YKh2AY&ved=0ahUKEwj1iaKmvIqEAxUFzAIHHXFBCGsQ4dUDCA0&uact=5&oq=9+news+australia&gs_lp=Egxnd3Mtd2l6LW5ld3MiEDkgbmV3cyBhdXN0cmFsaWEyCxAAGIAEGIoFGJECMgsQABiABBiKBRiRAjIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAESP4SUABY0BFwAHgAkAECmAG9BKAB8yWqAQkyLTkuMi4xLjK4AQPIAQD4AQHCAgsQABiABBixAxiDAcICDhAAGIAEGIoFGLEDGIMBwgINEAAYgAQYigUYsQMYCsICCBAAGIAEGLEDwgILEAAYgAQYigUYsQPCAhEQABiABBiKBRiRAhixAxiDAcICChAAGIAEGIoFGEPCAgcQABiABBgK&sclient=gws-wiz-news&start={num}'
         f'https://www.google.com.au/search?q=crikey+news&gl=au&hl=en&biw=1920&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=uLq7ZdfuEZLzi-gPnP21gAg&ved=0ahUKEwjXqvquvIqEAxWS-QIHHZx-DYAQ4dUDCA0&uact=5&oq=crikey+news&gs_lp=Egxnd3Mtd2l6LW5ld3MiC2NyaWtleSBuZXdzMgsQABiABBiKBRiRAjIFEAAYgAQyBRAAGIAEMgUQABiABDIHEAAYgAQYCjIGEAAYFhgeMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeMgYQABgWGB5I3xBQAFjaD3AAeACQAQCYAeYCoAGeFaoBBTItNi4zuAEDyAEA-AEBwgIKEAAYgAQYigUYQ8ICDRAAGIAEGIoFGEMYsQPCAgsQABiABBixAxiDAcICCBAAGIAEGLEDwgIOEAAYgAQYigUYsQMYgwHCAg4QABiABBiKBRiRAhixA8ICERAAGIAEGIoFGJECGLEDGIMBwgIQEAAYgAQYigUYQxixAxiDAcICChAAGIAEGAoYsQPCAg0QABiABBgKGLEDGIMBwgILEAAYgAQYigUYhgM&sclient=gws-wiz-news&start={num}',
         f'https://www.google.com.au/search?q=daily+telegraph+australia&gl=au&hl=en&biw=1920&bih=911&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=2bq7ZdiDKajYi-gPpeq7QA&ved=0ahUKEwiY1O--vIqEAxUo7AIHHSX1DggQ4dUDCA0&uact=5&oq=daily+telegraph+australia&gs_lp=Egxnd3Mtd2l6LW5ld3MiGWRhaWx5IHRlbGVncmFwaCBhdXN0cmFsaWEyCxAAGIAEGIoFGJECMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyCxAAGIAEGIoFGIYDSPo_UABY6TlwBXgAkAEEmAGZBqABvFOqAQ0yLTE0LjEwLjMuMS4yuAEDyAEA-AEBwgIIEAAYgAQYsQPCAgsQABiABBixAxiDAcICChAAGIAEGIoFGEPCAg4QABiABBiKBRixAxiDAcICEBAAGIAEGIoFGEMYsQMYgwHCAgcQABiABBgK&sclient=gws-wiz-news&start={num}'
    ]
    canada_urls=[
         f'https://www.google.ca/search?q=theglobeandmail.com&gl=ca&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=Mru7ZcH1N8KGxc8PysSN0Aw&ved=0ahUKEwjB1rbpvIqEAxVCQ_EDHUpiA8oQ4dUDCA0&uact=5&oq=theglobeandmail.com&gs_lp=Egxnd3Mtd2l6LW5ld3MiE3RoZWdsb2JlYW5kbWFpbC5jb21I2BdQ5ghYlRVwAHgAkAEAmAHZA6AB8hCqAQcyLTIuMi4yuAEDyAEA-AEBwgIEEAAYHsICCxAAGIAEGIoFGIYDiAYB&sclient=gws-wiz-news&start={num}',
         f'https://www.google.ca/search?q=thestar.com&gl=ca&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=Oru7ZdXbLuuXxc8PldqSqAE&ved=0ahUKEwjV4JXtvIqEAxXrS_EDHRWtBBUQ4dUDCA0&uact=5&oq=thestar.com&gs_lp=Egxnd3Mtd2l6LW5ld3MiC3RoZXN0YXIuY29tMgsQABiABBiKBRiRAjIFEAAYgAQyCxAAGIAEGIoFGJECMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgcQABiABBgKSLkYUL4KWJYWcAB4AJABAZgBngSgAaMjqgEJMi05LjQuMC4xuAEDyAEA-AEBwgIEEAAYHsICBhAAGB4YD8ICChAAGIAEGIoFGEPCAgsQABiABBixAxiDAcICDhAAGIAEGIoFGLEDGIMBwgIIEAAYgAQYsQPCAgQQABgDwgINEAAYgAQYChixAxiDAcICChAAGIAEGAoYsQOIBgE&sclient=gws-wiz-news&start={num}',
         f'https://www.google.ca/search?q=nationalpost.com&gl=ca&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=TLu7ZdjaIIXw6AO05KGYCA&ved=0ahUKEwjYsNL1vIqEAxUFOHoKHTRyCIMQ4dUDCA0&uact=5&oq=nationalpost.com&gs_lp=Egxnd3Mtd2l6LW5ld3MiEG5hdGlvbmFscG9zdC5jb20yCxAAGIAEGIoFGJECMgUQABiABDIHEAAYgAQYCjIFEAAYgAQyBRAAGIAEMgQQABgeMgQQABgeMgYQABgeGAoyBBAAGB4yBBAAGB5IqSZQrwlYniNwAHgAkAEBmAGbBaABxTSqAQkyLTcuOC4yLjK4AQPIAQD4AQHCAgoQABiABBiKBRhDwgILEAAYgAQYsQMYgwHCAgQQABgDwgIOEAAYgAQYigUYsQMYgwHCAggQABiABBixA8ICChAAGIAEGMcDGAqIBgE&sclient=gws-wiz-news&start={num}',
         f'https://www.google.ca/search?q=https%3A%2F%2Fottawacitizen.com&gl=ca&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=aru7Ze2aGLaMxc8PnYyg2Ac&ved=0ahUKEwjt9_CDvYqEAxU2RvEDHR0GCHsQ4dUDCA0&oq=https%3A%2F%2Fottawacitizen.com&gs_lp=Egxnd3Mtd2l6LW5ld3MiGWh0dHBzOi8vb3R0YXdhY2l0aXplbi5jb20yBRAAGIAEMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeSMYRULIDWLIDcAB4AJABAJgB_QKgAaMFqgEFMi0xLjG4AQzIAQD4AQL4AQHCAgsQABiABBiKBRiRAsICBxAAGIAEGArCAgQQABgewgIGEAAYHhgKiAYB&sclient=gws-wiz-news&start={num}',
         f'https://www.google.ca/search?q=trontosun.com&gl=ca&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=nbu7Zen5I_SXxc8P2bG2qAs&ved=0ahUKEwipvKWcvYqEAxX0S_EDHdmYDbUQ4dUDCA0&uact=5&oq=trontosun.com&gs_lp=Egxnd3Mtd2l6LW5ld3MiDXRyb250b3N1bi5jb20yBxAAGIAEGA0yBxAAGIAEGA0yBxAAGIAEGA0yBxAAGIAEGA0yBxAAGIAEGA0yBxAAGIAEGA0yBhAAGB4YDTIIEAAYHhgNGAoyBhAAGB4YDTIGEAAYHhgNSO8eUPMHWLwdcAB4AJABAZgB2AOgAcAkqgEIMi0xMi4yLjG4AQPIAQD4AQHCAgUQABiABMICBBAAGB7CAgYQABgWGB7CAgsQABiABBiKBRiRAsICCxAAGIAEGLEDGIMBwgIIEAAYgAQYsQPCAg4QABiABBiKBRixAxiDAcICEBAAGIAEGIoFGLEDGIMBGArCAgcQABiABBgKwgIKEAAYgAQYChixA8ICDRAAGIAEGAoYsQMYgwHCAg4QABiABBiKBRiRAhixA8ICERAAGIAEGIoFGJECGLEDGIMBwgINEAAYgAQYigUYQxixA8ICCRAAGIAEGA0YCogGAQ&sclient=gws-wiz-news&start={num}',
         f'https://www.google.ca/search?q=vancouverson.com&gl=ca&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=rbu7ZfevNYTw6APHyIjIBg&ved=0ahUKEwi3uoekvYqEAxUEOHoKHUckAmkQ4dUDCA0&uact=5&oq=vancouverson.com&gs_lp=Egxnd3Mtd2l6LW5ld3MiEHZhbmNvdXZlcnNvbi5jb20yBxAAGIAEGA0yBxAAGIAEGA0yBxAAGIAEGA0yBxAAGIAEGA0yBxAAGIAEGA0yBxAAGIAEGA0yBhAAGB4YDTIGEAAYHhgNMgYQABgeGA0yCBAAGB4YDRgKSOkaUABY4hhwAHgAkAECmAHvA6ABvSuqAQgyLTEyLjIuM7gBA8gBAPgBAcICDhAAGIAEGIoFGJECGLEDwgIREAAYgAQYigUYkQIYsQMYgwHCAggQABiABBixA8ICDhAAGIAEGIoFGLEDGIMBwgILEAAYgAQYsQMYgwHCAgUQABiABMICCxAAGIAEGIoFGLEDwgIHEAAYgAQYCsICCRAAGIAEGA0YCg&sclient=gws-wiz-news&start=%7Bnum%7D&start={num}'
    ]    
    compiled_urls = [uk_urls,usa_urls,canada_urls,australia_urls]
    for i,x in enumerate(compiled_urls):
        for b,url in enumerate(x):
            print(url)
            if '&gl=uk&' in url:
                scrape_and_process(url,'UK')
            if '&gl=us&' in url:
                scrape_and_process(url,'USA')
            if '&gl=au&' in url:
                scrape_and_process(url,'Australia')
            if '&gl=ca&' in url:
                scrape_and_process(url,'Canada')

0
https://www.google.co.uk/search?q=DAILY+MAIL+UK+NEWS&gl=uk&hl=en&tbs=qdr%3Ad%2Csbd%3A1&tbm=nws&ei=erG7Za6bD6GGxc8PmO-qqAg&ved=0ahUKEwiuwN7Gs4qEAxUhQ_EDHZi3CoUQ4dUDCA4&oq=DAILY+MAIL+UK+NEWS&gs_lp=Egxnd3Mtd2l6LW5ld3MiEkRBSUxZIE1BSUwgVUsgTkVXUzIFEAAYgAQyCxAAGIAEGIoFGJECMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeSJGUAVDAAljeiQFwAXgAkAEAmAHNAqAB0xCqAQUyLTIuNbgBDMgBAPgBAvgBAcICEBAAGIAEGIoFGEMYsQMYgwHCAgoQABiABBiKBRhDwgIGEAAYFhgewgIKEAAYCBgHGB4YCsICBBAAGB6IBgE&sclient=gws-wiz-news&start=0
https://www.dailymail.co.uk/tvshowbiz/article-13034911/justin-timberlake-britney-spears-apologizes-fans-concert.html
getting response
reponse passed getting article from url 
https://www.dailymail.co.uk/news/article-13037289/woman-animal-cruelty-dog-bin-trial.html
getting response
reponse passed getting article from url 
https://www.dailymail.co.uk/news/article-13037331/Bollywood-model-Poonam-Pandey-dies-aged-32-cervical-cancer-battle.html
getting response
reponse passe